In [118]:
from pyspark.sql import SparkSession


# Initialize Spark Session
spark = SparkSession.builder.appName("cycles").getOrCreate()

In [119]:
df_person = spark.read.csv("../workshop-files/person.csv", header=True)
df_products = spark.read.csv("../workshop-files/products.csv", header=True)
df_sales = spark.read.csv("../workshop-files/sales.csv", header=True)

In [120]:
from pyspark.sql.functions import col, udf, when

df_sales_details = (
    df_sales.join(df_products, "ProductID", "inner")
    .select(
        col("PersonID"),
        col("ProductID"),
        col("Model"),
        col("ProductCategoryName"),
        col("Region"),
)   .filter(
    (col('Model') == 'Bikes') | (col('ProductCategoryName') == 'Bikes')
))


df_cust_details = df_sales_details.join(df_person, "PersonID", "inner").drop("AddressLine2", "Title")

In [121]:
df_cust_details.show()

+--------+---------+--------------+-------------------+-------------+---------+----------+---------+---------+--------------------+------+--------------------+--------------------+-------------------+
|PersonID|ProductID|         Model|ProductCategoryName|       Region|FirstName|MiddleName| LastName|NameStyle|        Demographics|Suffix|        EmailAddress|        AddressLine1|        PhoneNumber|
+--------+---------+--------------+-------------------+-------------+---------+----------+---------+---------+--------------------+------+--------------------+--------------------+-------------------+
|    4937|      781|  Mountain-200|              Bikes|North America|    Devin|      NULL| Thompson|    False|"""<IndividualSur...|  NULL|devin14@adventure...|      618 Oak Street|       681-555-0150|
|    4937|      783|  Mountain-200|              Bikes|North America|    Devin|      NULL| Thompson|    False|"""<IndividualSur...|  NULL|devin14@adventure...|      618 Oak Street|       681-555-0

In [122]:
# Code by Diko changed the TotalPurhaseYTD column type to integer
from pyspark.sql.types import (
    StructType,
    StructField,
    StringType,
    IntegerType,
    DecimalType,
)
import xml.etree.ElementTree as ET

# Define the schema for the UDF's return type
schema = StructType(
    [
        StructField("TotalPurchaseYTD", DecimalType(), True),
        StructField("DateFirstPurchase", StringType(), True),
        StructField("BirthDate", StringType(), True),
        StructField("MaritalStatus", StringType(), True),
        StructField("YearlyIncome", StringType(), True),
        StructField("Gender", StringType(), True),
        StructField("TotalChildren", IntegerType(), True),
        StructField("NumberChildrenAtHome", IntegerType(), True),
        StructField("Education", StringType(), True),
        StructField("Occupation", StringType(), True),
        StructField("HomeOwnerFlag", IntegerType(), True),
        StructField("NumberCarsOwned", IntegerType(), True),
        StructField("CommuteDistance", StringType(), True),
    ]
)


def parse_xml(xml_str):
    if xml_str is None:
        return (None,) * 13

    xml_str = xml_str.strip('"').replace('"""', '"')
    xml_str = xml_str.replace('""', '"')

    if not xml_str.startswith("<") or not xml_str.endswith(">"):
        return (None,) * 13

    try:
        root = ET.fromstring(xml_str)
        namespace = {
            "ns": "http://schemas.microsoft.com/sqlserver/2004/07/adventure-works/IndividualSurvey"
        }
        return (
            root.findtext("ns:TotalPurchaseYTD", default="", namespaces=namespace),
            root.findtext("ns:DateFirstPurchase", default="", namespaces=namespace),
            root.findtext("ns:BirthDate", default="", namespaces=namespace),
            root.findtext("ns:MaritalStatus", default="", namespaces=namespace),
            root.findtext("ns:YearlyIncome", default="", namespaces=namespace),
            root.findtext("ns:Gender", default="", namespaces=namespace),
            int(root.findtext("ns:TotalChildren", default=0, namespaces=namespace)),
            int(
                root.findtext(
                    "ns:NumberChildrenAtHome", default=0, namespaces=namespace
                )
            ),
            root.findtext("ns:Education", default="", namespaces=namespace),
            root.findtext("ns:Occupation", default="", namespaces=namespace),
            int(root.findtext("ns:HomeOwnerFlag", default=0, namespaces=namespace)),
            int(root.findtext("ns:NumberCarsOwned", default=0, namespaces=namespace)),
            root.findtext("ns:CommuteDistance", default="", namespaces=namespace),
        )
    except Exception:
        raise ValueError(f"Unable to parse XML: {xml_str}")


parse_xml_udf = udf(parse_xml, schema)

df_cust_details = df_cust_details.withColumn(
    "expanded", parse_xml_udf(col("Demographics"))
)

for field in schema.fields:
    df_cust_details = df_cust_details.withColumn(
        field.name, col("expanded." + field.name)
    )


df_cust_details = df_cust_details.select(
    col("FirstName"),
    col("LastName"),
    col("AddressLine1"),
    col("DateFirstPurchase"),
    col("BirthDate"),
    col("MaritalStatus"),
    col("YearlyIncome"),
    col("Gender"),
    col("TotalChildren"),
    col("NumberChildrenAtHome"),
    col("Education"),
    col("Occupation"),
    col("HomeOwnerFlag"),
    col("NumberCarsOwned"),
    col("CommuteDistance"),
    col("TotalPurchaseYTD"),
    col("Model"),
    col("ProductCategoryName"),
)

In [123]:
df_cust_details.show()

+---------+--------+--------------------+-----------------+-----------+-------------+-------------------+------+-------------+--------------------+-------------------+--------------+-------------+---------------+---------------+----------------+------------+-------------------+
|FirstName|LastName|        AddressLine1|DateFirstPurchase|  BirthDate|MaritalStatus|       YearlyIncome|Gender|TotalChildren|NumberChildrenAtHome|          Education|    Occupation|HomeOwnerFlag|NumberCarsOwned|CommuteDistance|TotalPurchaseYTD|       Model|ProductCategoryName|
+---------+--------+--------------------+-----------------+-----------+-------------+-------------------+------+-------------+--------------------+-------------------+--------------+-------------+---------------+---------------+----------------+------------+-------------------+
|    Chloe|   Young|244 Willow Pass Road|      2003-07-22Z|1979-02-27Z|            S|        25001-50000|     F|            0|                   0|    Partial Coll

In [124]:
output_path = "../workshop-files/customer_details.csv"

df_cust_details.toPandas().to_csv(output_path, index=False, header=True)